In [ ]:
from typing import Annotated, TypedDict, Literal
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate
from IPython.display import display, Image
from dotenv import load_dotenv

import os

load_dotenv()  # Load environment variables from .env file
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

In [ ]:
class State(TypedDict):
    language: str
    messages: Annotated[list, add_messages]

def python_llm(state: State):
    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
    prompt = ChatPromptTemplate.from_messages({
        ("system", "You are a helpful assistant that writes Python code. return only the code for the requirements in the conversation, do not include any explanations."),
        ("user", "{messages}")
    })
    chain = prompt | llm
    response = chain.invoke({ "messages" : state["messages"] })
    return response.content

def js_llm(state: State):
    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
    prompt = ChatPromptTemplate.from_messages({
        ("system", "You are a helpful assistant that writes Typescript code. return only the code for the requirements in the conversation, do not include any explanations."),
        ("user", "{messages}")
    })
    chain = prompt | llm
    response = chain.invoke({ "messages" : state["messages"] })
    return response.content

def choose_language_llm(state: State) -> Literal['python_llm', 'js_llm']:
    if state["language"].lower() == "python":
        return "python_llm"
    else:
        return "js_llm"

def start_chat(state: State):
    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
    prompt = ChatPromptTemplate.from_messages({
        ("system", "You are a helpful assistant that helps users choose a programming language based on their needs."),
        ("user", "{messages}")
    })
    chain = prompt | llm
    response = chain.invoke({ "messages" : state["messages"] })
    state["language"] = "Python" if "python" in response.content.lower() else "Typescript"
    return choose_language_llm(state)

In [ ]:
graph = StateGraph(State)
graph.add_node("start_chat", start_chat)
graph.add_node("python_llm", python_llm)
graph.add_node("js_llm", js_llm)


graph.add_edge(START, "start_chat")
graph.add_conditional_edges("start_chat", choose_language_llm)
graph.add_edge("js_llm",END)
graph.add_edge("python_llm",END)

graph_builder = graph.compile()


display(Image(graph_builder.get_graph().draw_mermaid_png()))

In [ ]:
graph_builder.invoke({
    "messages": [
        {"role": "user", "content": "I want to wrote a function in javascript"}
    ]
})